In [100]:
import pandas as pd
import numpy as np
import praw
from datetime import datetime
import re
from psaw import PushshiftAPI

In [4]:
reddit = praw.Reddit(client_id='99VP5wAYO4IWtA', client_secret='IFJ2dvaOMcFR6RILmHuk0HYgieLyqg', user_agent='wsb')
api = PushshiftAPI(reddit)

In [213]:
def get_new_posts():
    start_epoch=int(datetime(2021, 1, 20).timestamp())
    new_posts = list(api.search_submissions(subreddit='wallstreetbets',
                            after=start_epoch,
                            sort='desc',))
    posts = pd.DataFrame(columns=['id', 'date', 'author', 'title', 'comments', 'num_comments', 'score', 'ratio', 'url'])
    x = 0
    for post in new_posts:
        post.comments.replace_more(limit=0)
        posts.loc[x] = [post.id, 
                        datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                        post.author, 
                        post.title, 
                        [
                            {'body': j.body, 'author': j.author, 'score': j.score, 'date': datetime.utcfromtimestamp(j.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 'link': j.permalink}
                            for j in post.comments.list()
                        ],
                        post.num_comments,
                        post.score,
                        post.upvote_ratio,
                        post.permalink
                       ]
        x += 1
    return posts

In [ ]:
posts = get_new_posts()

In [ ]:
posts

In [92]:
def get_stocks():
    file = open("nasdaqtraded.txt", "r")
    columns = file.readline()[:-2].split('|')
    current_stocks = pd.DataFrame(columns=columns)
    x = 0
    for line in file:
        try:
            row = line[:-2].split('|')
            current_stocks.loc[x] = row
            x += 1
        except:
            print(row)
    return current_stocks

In [86]:
current_stocks.to_csv('current_stocks.csv')

In [89]:
tickers = current_stocks['Symbol']

In [144]:
gme = pd.DataFrame(columns=['date'])
x = 0
y = 0
for row in posts['title']:
    row = re.sub(r'[^A-Za-z0-9 ]+', ' ', row)
    for word in (row.split(' ')):
        if word.upper() in ['GME']:
            gme.loc[y] = posts['date'][x]
            y += 1
    x += 1
    


In [147]:
posts['comments'][1][0]

{'body': 'Sell limit 1000',
 'author': Redditor(name='high_class_autist'),
 'score': 3,
 'date': '2021-01-24 03:41:27',
 'link': '/r/wallstreetbets/comments/l3rodx/gme_exit_strategies/gkhnq7o/'}

In [148]:
for comments in posts['comments']:
    for comment in comments:
        c = re.sub(r'[^A-Za-z0-9 ]+', ' ', (comment['body']))
        for word in (c.split(' ')):
            if word.upper() in ['GME']:
                gme.loc[y] = comment['date']
                y += 1

In [154]:
gme['date'] = gme['date'].astype('datetime64[ns, US/Pacific]')

In [158]:
gme.groupby(gme['date'].dt.day).agg(['count'])

,date
,count
date,
22,3066
23,2380


GME Exit Strategies
